In [ ]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

In [ ]:
clientes = spark.read.load("dados/Clientes.parquet")
vendas = spark.read.load("dados/Vendas.parquet")

In [ ]:
from pyspark.sql import functions as Func

In [ ]:
#1 Crie uma consulta que mostre, nesta ordem, Cliente, Estados e Status
clientes.select("Cliente","Estado","Status").show()

In [ ]:
#2 Crie uma consulta que mostre apenas os clientes do Status “platinum” e “gold”
clientes.select("*").where( (Func.col("Status") == "Gold") | (Func.col("Status") == "Platinum") ).show()

In [ ]:
#3 Demostre quanto cada Status de Clientes representa em vendas?
vendas.join(clientes,vendas.ClienteID ==clientes.ClienteID ).groupBy(clientes.Status).agg(sum("Total")).orderBy(Func.col("sum(Total)").desc())